In this homework, you will debias word embeddings using the method from [Bolukbasi et al. 2016](https://arxiv.org/abs/1607.06520) and interpreted through [Vargas and Cotterell 2020](https://arxiv.org/abs/2009.09435). 

In [ ]:
import re
from gensim.models import KeyedVectors
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
glove = KeyedVectors.load_word2vec_format("../data/glove.6B.100d.100K.w2v.txt", binary=False)

In [ ]:
# let's print one sample vector just to see what it looks like
print(glove["man"].shape)
print(glove["man"])

Now let's calculate the cosine similarity of that vector ("man") with a set of other vectors ("king" and "cabbage").  This returns two cosine similarities, the first cos(man, king) and the second cos(man, cabbage).

In [ ]:
glove.cosine_similarities(glove["man"], [glove["king"], glove["cabbage"]])

Let's use that machinery to find the differences between "man" and "woman" and a set of target terms.

In [ ]:
targets=["doctor", "nurse", "actor", "actress", "mechanic", "librarian", "architect", "magician", "cook", "chef"]
diffs={}
for term in targets:
    
    m,w=glove.cosine_similarities(glove[term], [glove["man"], glove["woman"]])
    diffs[term]=m-w

for k, v in sorted(diffs.items(), key=lambda item: item[1], reverse=True):
    print("%.3f\t%s" % (v,k))

We can see a gender difference here, where "man" is more aligned "magician" and "mechanic" and "woman" is more aligned with "actress" and "nurse".

**Q1.** Let's debias those embeddings, using the method from [Bolukbasi et al. 2016](https://arxiv.org/abs/1607.06520) and interpreted through [Vargas and Cotterell 2020](https://arxiv.org/abs/2009.09435).  Debiasing embeddings requires two steps: finding the gender subspace and then subtracting the orthogonal projection onto that subspace from the original embedding.  Let's start with the first step: creating "defining sets" that capture the variation:

$$
D_1 = \{man, woman\}\\
D_2 = \{mr., mrs.\}
$$

Following Vargas and Cotterell, we can find the gender subspace by constructing a new matrix $D'$ by substracting the embedding for a word in a defining set from the average of embeddings in that set. Using $e_{word}$ to denote the embedding for a word, this process would results in the following for the defining sets above:

$$
\begin{bmatrix}
e_{man} - \textrm{mean}(e_{man},e_{woman}) \\
e_{woman} - \textrm{mean}(e_{man},e_{woman})\\
e_{mr.} - \textrm{mean}(e_{mr.},e_{mrs.})\\
e_{mrs.} - \textrm{mean}(e_{mr.},e_{mrs.})\\
\end{bmatrix}
$$

If the original embeddings (e.g., for $e_{man}$) are 100 dimensions (and so the mean over any set of embeddings is also 100 dimensions), then the resulting matrix $D'$ should be $4 \times 100$.  Create this matrix $D'$ and name it `subspace_matrix`.

In [ ]:
D1=["man", "woman"]
D2=["mr.", "mrs."]

# TODO   

In [ ]:
# This should be (4,100)
print(subspace_matrix.shape)

Step two is to run [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) over that `subspace_matrix` matrix.  The gender subspace in this example is the first principle component of that process. Here's how you run PCA on a random matrix to get the first principle component.

In [ ]:
fake_matrix=np.random.rand(3,3)
print("fake matrix:")
print(fake_matrix)

# We only need one principle component, so we'll set n_components=1
pca=PCA(n_components=1).fit(fake_matrix)
subspace=pca.components_[0]

print("first principle component:")
print(subspace)

In [ ]:
# You'll see that this subspace is already normalized to unit length:
print(subspace)
print(subspace/np.sqrt(np.dot(subspace, subspace)))

**Q2.** Run PCA on that subspace matrix to get the subspace axis.

That subspace is the gender axis. You'll remember from class that we find the orthogoal projection of any unit-normalized vector $w$ onto a subspace $b$ by:

$$
w_b = \textrm{dot}(w,b) \; b
$$

If $b$ and $x$ are 100 dimensions, $w_b$ is 100 dimensions too.  The debiased vector $w_d$ is then simply $w - w_b$.  

**Q3.** Debias the vectors for "man", "woman", and the targets used above ("doctor", "nurse", "actor", "actress", "mechanic", "librarian", "architect", "magician", "cook", "chef") and see if debiasing changes the differences between these terms and "man"/"woman" as noted above.  Glove embeddings are not normalized ahead of time, so be sure to normalize them before carrying out your projection (i.e., dividing vector v by $\sqrt{\textrm{dot}(v,v)}$).


**check-plus**. Reflect in 100 words on the differences between this gender axis construction and the axis construction in SemAxis.  How are they different?